## Ensemble learning for activity recognition

### 1. Set libraries

In [55]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from matplotlib import pyplot
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Activation
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from random import randint
from numpy import array
from numpy import argmax
from pandas import DataFrame
from pandas import concat
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import Concatenate 
from tensorflow.keras.layers import Dense, concatenate
#from tensorflow.keras_self_attention import SeqSelfAttention

### 1. Load the data

In [75]:
length=495
# load dataset with pandas
dataframe1 = pd.read_csv('data/input_v2/Presentation_activity.csv', sep=',', header=None)
dataframe2 = pd.read_csv('data/input_v2/Chatting_activity.csv', sep=',', header=None)
dataframe3 = pd.read_csv('data/input_v2/Discussion_activity.csv', sep=',', header=None)
dataframe4 = pd.read_csv('data/input_v2/GroupStudy_activity.csv', sep=',', header=None)

# Show how many episodes 
#print(len(dataframe1))
#print(len(dataframe2))
#print(len(dataframe3))
#print(len(dataframe4))

#Combine all dataframe
dataframe = dataframe1.append(dataframe2,ignore_index=True)
dataframe = dataframe.append(dataframe3,ignore_index=True)
dataframe = dataframe.append(dataframe4,ignore_index=True)

#Change NaN data to 0
dataframe = dataframe.fillna(0)
dataset = dataframe.values

#288, 6
#Seperate input and do one-hot encoding
X=to_categorical(dataset)
print("The shape of X:", X.shape)

# Construct Y label
arr = [] #  empty regular list
for i in range(len(dataframe1)):
    arr.append(0*np.ones((1)))
for i in range(len(dataframe2)):
    arr.append(1*np.ones((1)))
for i in range(len(dataframe3)):
    arr.append(2*np.ones((1)))
for i in range(len(dataframe4)):
    arr.append(3*np.ones((1)))
np_array = np.array(arr)  # transformed to a numpy array
y_df = pd.DataFrame({'': np_array[:, 0]})

Y=to_categorical(y_df)
print("The shape of Y:", Y.shape)


# Divide train and test data
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2)

The shape of X: (340, 724, 8)
The shape of Y: (340, 4)
